# Grain Boundary 3D FCC Metals (Copper)

## 0. Introduction

This notebook demonstrates how to create a grain boundary structure for a 3D FCC metal, Cu in particular. The grain boundary is created by combining two slabs with different orientations. 

Aiming to reproduce the structure from the publication:
> **Timofey Frolov, David L. Olmsted, Mark Asta & Yuri Mishin**
> "Structural phase transformations in metallic grain boundaries"
> Nature Communications, volume 4, Article number: 1899 (2013)
> DOI: [10.1038/ncomms2919](https://www.nature.com/articles/ncomms2919) 

Focusing on reproducing the structures from Figure 1. b:

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/defects/defect_planar_grain_boundary_3d_fcc_metal/0-figure-from-manuscript.webp" alt="Figure 1." width="400"/>

## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [19]:


# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False
MATERIAL_NAME = "Cu"

# Parameters for Phase 1
PHASE_1_MILLER_INDICES = (3, 1, 0)
PHASE_1_THICKNESS = 4  # in atomic layers
PHASE_1_USE_ORTHOGONAL_C = True

# Parameters for Phase 2
PHASE_2_MILLER_INDICES = (-3, -1, 0)
PHASE_2_THICKNESS = 4  # in atomic layers
PHASE_2_USE_ORTHOGONAL_C = True

INTERPHASE_GAP = 2.0  # in Angstrom

# Maximum area for the superlattice search algorithm
MAX_AREA = 100  # in Angstrom^2

# Parameters for the final material
SLAB_MILLER_INDICES = (0, 0, 1)
SLAB_THICKNESS = 4  # in atomic layers
SLAB_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
SLAB_VACUUM = 20.0  # in Angstrom

# Set the termination pair indices
TERMINATION_PAIR_INDEX = 0  # Will be overridden if interactive selection is used

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [20]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples|create_grain_boundary.ipynb")


### 1.3. Get input material

In [21]:
from mat3ra.standata.materials import Materials
from mat3ra.made.tools.build import MaterialWithBuildMetadata

material = MaterialWithBuildMetadata.create(Materials.get_by_name_first_match(MATERIAL_NAME))

### 1.4. Preview Material

In [22]:
from utils.visualize import visualize_materials as visualize

visualize([material], repetitions=[3, 3, 1], rotation="0x")

GridBox(children=(VBox(children=(Label(value='Cu - Material - rotation: 0x', layout=Layout(align_self='center'…

## 2. Create grain boundary

### 2.1. Create Phase 1 and Phase 2 Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.

### 2.2. Get possible terminations for the slabs

In [23]:
from mat3ra.made.tools.build.slab.helpers import get_slab_terminations

phase_1_terminations = get_slab_terminations(material=material, miller_indices=PHASE_1_MILLER_INDICES)
phase_2_terminations = get_slab_terminations(material=material, miller_indices=PHASE_2_MILLER_INDICES)

print(f"Phase 1 Terminations:")
for idx, termination in enumerate(phase_1_terminations):
    print(f"    {idx}: {termination}")
print(f"Phase 2 Terminations:")
for idx, termination in enumerate(phase_2_terminations):
    print(f"    {idx}: {termination}")

Phase 1 Terminations:
    0: Cu_P2/m_2
Phase 2 Terminations:
    0: Cu_P2/m_2


### 2.3. Visualize slabs for all possible terminations

In [24]:
from mat3ra.made.tools.build.slab.helpers import create_slab

phase_1_slabs = [create_slab(material, miller_indices=PHASE_1_MILLER_INDICES, termination=termination) for termination
                 in phase_1_terminations]
phase_2_slabs = [create_slab(material, miller_indices=PHASE_2_MILLER_INDICES, termination=termination) for termination
                 in phase_2_terminations]

visualize([{"material": slab, "title": slab.name} for slab in phase_1_slabs],
          repetitions=[3, 3, 1], rotation="-90x")
visualize([{"material": slab, "title": slab.name} for slab in phase_2_slabs],
          repetitions=[3, 3, 1], rotation="-90x")

GridBox(children=(VBox(children=(Label(value='Cu12 - Cu(310), termination Cu_P2/m_2, Slab - rotation: -90x', l…

GridBox(children=(VBox(children=(Label(value='Cu12 - Cu(-3̂-1̂0), termination Cu_P2/m_2, Slab - rotation: -90x…

In [25]:


# Create phase 1 slab
phase_1_slab = create_slab(
    crystal=material,
    termination=None,  # Use default termination
    miller_indices=PHASE_1_MILLER_INDICES,
    number_of_layers=PHASE_1_THICKNESS,
    vacuum=0.0,
    use_orthogonal_c=PHASE_1_USE_ORTHOGONAL_C
)

# Create phase 2 slab  
phase_2_slab = create_slab(
    crystal=material,
    termination=None,  # Use default termination
    miller_indices=PHASE_2_MILLER_INDICES,
    number_of_layers=PHASE_2_THICKNESS,
    vacuum=0.0,
    use_orthogonal_c=PHASE_2_USE_ORTHOGONAL_C
)

# Visualize the phase slabs
visualize([{"material": phase_1_slab, "title": "Phase 1 Slab"}, {"material": phase_2_slab, "title": "Phase 2 Slab"}],
          repetitions=[2, 2, 1], rotation="-90x")


/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/mat3ra/made/lattice.py:91: RuntimeWarning: invalid value encountered in scalar divide
  alpha = np.degrees(np.arccos(np.dot(vectors[1], vectors[2]) / (b * c)))
/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/mat3ra/made/lattice.py:92: RuntimeWarning: invalid value encountered in scalar divide
  beta = np.degrees(np.arccos(np.dot(vectors[0], vectors[2]) / (a * c)))
/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected `AtomicLayersUniqueConfiguration` but got `AtomicLayersUniqueRepeatedConfiguration` with value `AtomicLayersUniqueRepeate...number_of_repetitions=4)` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `Material` but got `MaterialWithBuildMetadata` with value `MaterialWithBuildMetadata..

GridBox(children=(VBox(children=(Label(value='Cu48 - Phase 1 Slab - rotation: -90x', layout=Layout(align_self=…

### 2.4. Print terminations for the grain boundary

In [26]:
from itertools import product

termination_pairs = list(product(phase_1_terminations, phase_2_terminations))
print("Termination Pairs (Phase 1, Phase 2)")
for idx, termination_pair in enumerate(termination_pairs):
    print(f"    {idx}: {termination_pair}")

Termination Pairs (Phase 1, Phase 2)
    0: (Cu_P2/m_2, Cu_P2/m_2)


### 2.5. Select termination pair for the grain boundary

In [27]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

termination_pair_index = TERMINATION_PAIR_INDEX

termination_pair = termination_pairs[termination_pair_index]
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination_pair = await ui_prompt_select_array_element_by_index_pyodide(termination_pairs,
                                                                                 element_name="phase 1/phase 2 termination pair")
    else:
        termination_pair = ui_prompt_select_array_element_by_index(termination_pairs,
                                                                   element_name="phase 1/phase 2 termination pair")

### 2.6. Initialize the Grain Boundary Configuration

In [28]:
from mat3ra.made.tools.build.grain_boundary import SlabGrainBoundaryConfiguration

phase_1_termination, phase_2_termination = termination_pair

# Create a slab configuration for the final grain boundary structure
slab_configuration = SlabConfiguration(
    bulk=material,
    miller_indices=SLAB_MILLER_INDICES,
    number_of_layers=SLAB_THICKNESS,
    vacuum=SLAB_VACUUM,
)

grain_boundary_configuration = SlabGrainBoundaryConfiguration(
    phase_1_configuration=phase_1_configuration,
    phase_2_configuration=phase_2_configuration,
    phase_1_termination=phase_1_termination,
    phase_2_termination=phase_2_termination,
    gap=INTERPHASE_GAP,
    slab_configuration=slab_configuration
)

ImportError: cannot import name 'SlabGrainBoundaryConfiguration' from 'mat3ra.made.tools.build.grain_boundary' (/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/mat3ra/made/tools/build/grain_boundary/__init__.py)

### 2.7. Set Strain Matching Algorithm Parameters

In [ ]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingParameters
from mat3ra.made.tools.build.grain_boundary.builders import SlabGrainBoundaryBuilderParameters

zsl_strain_matching_parameters = ZSLStrainMatchingParameters(
    max_area=MAX_AREA
)

builder_parameters = SlabGrainBoundaryBuilderParameters(
    strain_matching_parameters=zsl_strain_matching_parameters
)


### 2.8. Generate grain boundary with strain matcher

In [ ]:
from mat3ra.made.tools.build.grain_boundary import SlabGrainBoundaryBuilder

grain_boundary_builder = SlabGrainBoundaryBuilder(build_parameters=builder_parameters)
grain_boundary = grain_boundary_builder.get_material(configuration=grain_boundary_configuration)

## 2.9. Preview the grain boundary

In [ ]:
visualize([grain_boundary], repetitions=[1, 1, 1], viewer="wave")

## 3. Save the final material

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials(grain_boundary)
download_content_to_file(grain_boundary,
                         f"{MATERIAL_NAME}-{PHASE_1_MILLER_INDICES}-{PHASE_2_MILLER_INDICES}_grain_boundary.json")